In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import ElasticNet, enet_path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import sys
import warnings
from itertools import cycle
warnings.filterwarnings("ignore")
np.random.seed(40)
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# Load Diabetes datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

# Create pandas DataFrame for sklearn ElasticNet linear_model
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = diabetes.feature_names + ["progression"]
data = pd.DataFrame(d, columns=cols)

# Evaluate metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "progression" which is a quantitative measure of disease progression one year after baseline
train_x = train.drop(["progression"], axis=1)
test_x = test.drop(["progression"], axis=1)
train_y = train[["progression"]]
test_y = test[["progression"]]

# Define the experiment name
experiment_name = "diaclf_experiment"

# Check if the experiment exists
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    # If the experiment does not exist, create it
    mlflow.create_experiment(experiment_name)

# Set the experiment
mlflow.set_experiment(experiment_name)

# Define the hyperparameters to try
alpha_values = [0.01, 0.05]
l1_ratio_values = [0.01, 0.05]

for alpha in alpha_values:
    for l1_ratio in l1_ratio_values:
        with mlflow.start_run(run_name=f"alpha_{alpha}_l1_ratio_{l1_ratio}"):
            # Run ElasticNet
            lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
            lr.fit(train_x, train_y)
            predicted_qualities = lr.predict(test_x)
            (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

            # Print out ElasticNet model metrics
            print(f"Elasticnet model (alpha={alpha:f}, l1_ratio={l1_ratio:f}):")
            print(f"  RMSE: {rmse}")
            print(f"  MAE: {mae}")
            print(f"  R2: {r2}")

            # Infer model signature
            predictions = lr.predict(train_x)
            signature = infer_signature(train_x, predictions)

            # Log mlflow attributes for mlflow UI
            mlflow.log_param("alpha", alpha)
            mlflow.log_param("l1_ratio", l1_ratio)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            mlflow.sklearn.log_model(lr, "model", signature=signature)

# end current run
mlflow.end_run()

# Get experiment ID
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

# Search all runs in experiment_id
runs = mlflow.search_runs(experiment_ids=experiment_id)

# display runs dataframe
runs


2023/12/11 13:00:56 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Elasticnet model (alpha=0.010000, l1_ratio=0.010000):
  RMSE: 71.40365347371498
  MAE: 60.09131524886582
  R2: 0.2291124645214172
Elasticnet model (alpha=0.010000, l1_ratio=0.050000):
  RMSE: 71.18704555534748
  MAE: 59.893223242091494
  R2: 0.2337824516449345
Elasticnet model (alpha=0.050000, l1_ratio=0.010000):
  RMSE: 78.71957310643826
  MAE: 66.42794905090514
  R2: 0.06305172320439234
Elasticnet model (alpha=0.050000, l1_ratio=0.050000):
  RMSE: 78.59249466381223
  MAE: 66.30998032458166
  R2: 0.06607434687959934


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.mae,metrics.r2,metrics.rmse,params.l1_ratio,params.alpha,tags.mlflow.log-model.history,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.source.name
0,7db322eacd624b10bed89627c41cd336,734078547406037714,FINISHED,file:///d:/praxis/Term%202/MLOP/MLFLOW/mlruns/...,2023-12-11 07:31:25.636000+00:00,2023-12-11 07:31:30.701000+00:00,66.309980,0.066074,78.592495,0.05,0.05,"[{""run_id"": ""7db322eacd624b10bed89627c41cd336""...",moazi,LOCAL,alpha_0.05_l1_ratio_0.05,d:\Anaconda\Conda\Lib\site-packages\ipykernel_...
1,b21a55c56e814479bd5ff6dd679bf326,734078547406037714,FINISHED,file:///d:/praxis/Term%202/MLOP/MLFLOW/mlruns/...,2023-12-11 07:31:19.586000+00:00,2023-12-11 07:31:25.624000+00:00,66.427949,0.063052,78.719573,0.01,0.05,"[{""run_id"": ""b21a55c56e814479bd5ff6dd679bf326""...",moazi,LOCAL,alpha_0.05_l1_ratio_0.01,d:\Anaconda\Conda\Lib\site-packages\ipykernel_...
2,5c7ba91bcaf942b794fd2166cb23248f,734078547406037714,FINISHED,file:///d:/praxis/Term%202/MLOP/MLFLOW/mlruns/...,2023-12-11 07:31:14.679000+00:00,2023-12-11 07:31:19.573000+00:00,59.893223,0.233782,71.187046,0.05,0.01,"[{""run_id"": ""5c7ba91bcaf942b794fd2166cb23248f""...",moazi,LOCAL,alpha_0.01_l1_ratio_0.05,d:\Anaconda\Conda\Lib\site-packages\ipykernel_...
3,b7b3387ac75548029988cb8061945897,734078547406037714,FINISHED,file:///d:/praxis/Term%202/MLOP/MLFLOW/mlruns/...,2023-12-11 07:30:56.761000+00:00,2023-12-11 07:31:14.668000+00:00,60.091315,0.229112,71.403653,0.01,0.01,"[{""run_id"": ""b7b3387ac75548029988cb8061945897""...",moazi,LOCAL,alpha_0.01_l1_ratio_0.01,d:\Anaconda\Conda\Lib\site-packages\ipykernel_...


In [2]:
# similar display is available in mlflow ui - select columns to show these metrics also